In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
BMJ_data_all__b_new = pd.read_csv('../input/BMJ-data-all--b-new.csv', sep='\t')
item_profiles1 = pd.read_csv('../input/item-profiles1.csv', sep=';')
item_profiles2 = pd.read_csv('../input/item-profiles2.csv', sep=';')
item_profiles3 = pd.read_csv('../input/item-profiles3.csv', sep=';')
ratings = pd.read_csv('../input/user-item-rating.csv', sep='\t', names=['user_id','item_id','rating'])

In [3]:
# We use this healthiness method to calculate the healthiness of recipes, and to set weights for the post filtering.

In [4]:
def healthiness(itemsDataframe):
    
    # Calculate the energy percentage of each relevant macronutrient. 1g of fat contains 9 kCal. 
    fat = ((itemsDataframe['Fat (g)'] * 9) / itemsDataframe['Calories (kCal)']) * 100
    sugar = ((itemsDataframe['Sugar (g)'] * 4) / itemsDataframe['Calories (kCal)']) * 100
    saturatedFat = ((itemsDataframe['Saturated Fat (g)'] * 9) / itemsDataframe['Calories (kCal)']) * 100
    
    # This calculates a continous version of the healthiness score. Fat/3 because the recommended limit for fat is 3X the others.
    itemsDataframe['Unhealtiness'] = (fat / 3) + sugar + saturatedFat
    
    # These are bounderies and points are tunable to influence 
    # post filter weights in accordence with the health recommendations.
    itemsDataframe.loc[fat > 30, 'fatPoints'] = 0
    itemsDataframe.loc[fat >= 40, 'fatPoints'] = -1
    itemsDataframe.loc[fat >= 50, 'fatPoints'] = -2
    itemsDataframe.loc[fat >= 60, 'fatPoints'] = -3
    itemsDataframe.loc[fat >= 70, 'fatPoints'] = -4
    itemsDataframe.loc[fat >= 80, 'fatPoints'] = -5
    itemsDataframe.loc[fat <= 30, 'fatPoints'] = 1 
    itemsDataframe.loc[fat <= 20, 'fatPoints'] = 2
    itemsDataframe.loc[fat <= 10, 'fatPoints'] = 3
    itemsDataframe.loc[fat <= 5, 'fatPoints'] = 4 
    itemsDataframe.loc[fat <= 1, 'fatPoints'] = 5

    itemsDataframe.loc[sugar > 10, 'sugarPoints'] = 0
    itemsDataframe.loc[sugar >= 13, 'sugarPoints'] = -1
    itemsDataframe.loc[sugar >= 16, 'sugarPoints'] = -2
    itemsDataframe.loc[sugar >= 19, 'sugarPoints'] = -3
    itemsDataframe.loc[sugar >= 22, 'sugarPoints'] = -4
    itemsDataframe.loc[sugar >= 25, 'sugarPoints'] = -5
    itemsDataframe.loc[sugar <= 10, 'sugarPoints'] = 1 
    itemsDataframe.loc[sugar <= 7, 'sugarPoints'] = 2
    itemsDataframe.loc[sugar <= 5, 'sugarPoints'] = 3 
    itemsDataframe.loc[sugar <= 3, 'sugarPoints'] = 4
    itemsDataframe.loc[sugar <= 1, 'sugarPoints'] = 5 
        
    itemsDataframe.loc[saturatedFat > 10, 'satFatPoints'] = 0
    itemsDataframe.loc[saturatedFat >= 13, 'satFatPoints'] = -1
    itemsDataframe.loc[saturatedFat >= 16, 'satFatPoints'] = -2
    itemsDataframe.loc[saturatedFat >= 19, 'satFatPoints'] = -3
    itemsDataframe.loc[saturatedFat >= 22, 'satFatPoints'] = -4
    itemsDataframe.loc[saturatedFat >= 25, 'satFatPoints'] = -5
    itemsDataframe.loc[saturatedFat <= 10, 'satFatPoints'] = 1 
    itemsDataframe.loc[saturatedFat <= 7, 'satFatPoints'] = 2
    itemsDataframe.loc[saturatedFat <= 5, 'satFatPoints'] = 3 
    itemsDataframe.loc[saturatedFat <= 3, 'satFatPoints'] = 4
    itemsDataframe.loc[saturatedFat <= 1, 'satFatPoints'] = 5 
        
       
    itemsDataframe['Healthiness'] = itemsDataframe['fatPoints'] + itemsDataframe['satFatPoints'] + itemsDataframe['sugarPoints']

In [5]:
# Apply healthiness evaluation to each recipe

In [6]:
healthiness(item_profiles2)
healthiness_profiles = item_profiles2[['Recipe ID', 'Healthiness']]

healthiness_dict = defaultdict()
for _,Recipe_ID, Healthiness in healthiness_profiles.itertuples():
    healthiness_dict[Recipe_ID] = Healthiness

In [21]:
item_profiles2['Healthiness'].mean()

-0.5383123181377304

In [7]:
# This is the post filter method. Suprise predictions are given in a list of immutable tuples.
# To post filter the predictions we make a dataframe of the predictions list, change the predictions,
# and make a new list of tuples.

In [8]:
def postfilter(predictions, healthinessFactor):
    ratingsProcessed = pd.DataFrame.from_records(predictions, columns=['uid', 'iid', 'r_ui', 'est', 'details'])
    ratingsProcessed = ratingsProcessed.join(item_profiles2['Healthiness'], how= 'inner', on= 'iid', sort=False)
    # This is were the values are transformed based on the healthinessFactor parameter and the health points of the recipes.
    ratingsProcessed['est'] = ratingsProcessed['est'] + (ratingsProcessed['Healthiness']*healthinessFactor)
    ratingsProcessed.drop(labels='Healthiness', axis=1, inplace=True)
    ratingsProcessed = list(ratingsProcessed.itertuples(name='Prediction', index=False))
    return ratingsProcessed

In [9]:
from collections import defaultdict
from sklearn.metrics import ndcg_score

def evaluations_at_k(predictions):

    # K is the number of highest ranking predictions to consider
    k = 10
    # This is the threshold for what is considered an adequate recommendation.
    threshold = 3.5
    
    # We map the predictions to the users. uid=user identity, iid=item identity, true_r[ating], est[imated rating]. 
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Sort the predictions for each user and retrieve the k highest ones and put them in a dictionary.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:k]
        
    # Remove the dictionary entries that didn't have enough ratings because of the data partitioning. 
    top_k = top_n.copy()
    for uid, user_ratings in top_k.items():
        if (len(top_k[uid]) != k):
            top_n.pop(uid)
            
    # Calculate the healthiness average of all recommendations. 
    # 0 = the tipping point between inside and outside of HDR recommendations. Negative value means unhealthy.
    healthinessAverage = 0
    count = 0
    for x in top_n.items():
        count =  count + 1
        healthiness = 0
        for y in x[1]:
            healthiness = healthiness + healthiness_dict[y[0]]
        healthinessAverage = healthinessAverage + (healthiness/k)
    
    healthinessAverage = healthinessAverage/count
    
    # This time we put put the estimated rating and the true rating in the dictionary.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
        
    # Extract relevant lists from the named tuples in predictions, and calculate NDCG.
    predictions_list = [[x[3] for x in predictions]]
    true_scores = [[x[2] for x in predictions]]
    ndcg = ndcg_score(true_scores, predictions_list, k)
    
    # Calculate the averages of precision and recall
    precision = (sum(prec for prec in precisions.values())) / len(precisions)
    recall = (sum(rec for rec in recalls.values())) / len(recalls)
    
    
    return precision, recall, ndcg, healthinessAverage, top_n
    

In [10]:
ratings

,user_id,item_id,rating
0,455,50,3.0
1,455,457,4.0
2,455,28,5.0
3,455,458,3.0
4,455,459,5.0
...,...,...,...
50676,84839,131,3.0
50677,84839,109,5.0
50678,84839,145,5.0
50679,84839,133,5.0


In [ ]:
    users = train_ratings['user_id'].drop_duplicates().to_frame()
    recipes = train_ratings['item_id'].drop_duplicates().to_frame()
    users['key'] = 1
    recipes['key'] = 1
    users_recipes = users.merge(recipes, how='outer').drop("key", 1)
    users_recipes['rating_est'] = np.nan
    
    reader = Reader()
    data = Dataset.load_from_df(train_ratings, reader)
    trainset = data.build_full_trainset()
    svd.fit(trainset)
    
    
    users_recipes['rating_est'] = users_recipes.apply(lambda x: svd.predict(x.user_id, x.recipe_id, r_ui=None, clip=True, verbose=False)[3], axis=1)
    
    
    #for user_id, item_id, rating in train_ratings.itertuples(name='ratings', index=False):
     #   users_recipes.loc[('user_id' == user_id) & ('recipe_id' == item_id)].at['rating_est'] = rating
    
        
    train_ratings = users_recipes


In [114]:
def content_based_predict(train, test):
    
    train_ratings = ratings.rename(columns={'item_id': 'recipe_id'}).iloc[train]


    
    recipe_id_ingredient_id = item_profiles3[['Recipe ID', 'Ingredient ID']]
    ingredientRatings = train_ratings.merge(recipe_id_ingredient_id, left_on='recipe_id', right_on='Recipe ID').drop('Recipe ID',1)
    ingredientRatings = ingredientRatings[['user_id', 'recipe_id', 'Ingredient ID', 'rating']]
    ingredientRatings = ingredientRatings.rename(columns={'Ingredient ID': 'ingredient_id', 'item_id': 'recipe_id', 'rating': 'rat'})

    recipes_dict = defaultdict(list) 
    ingredients_dict = defaultdict(lambda: defaultdict(list))  
    for user_id, recipe_id, ingredient_id, rating in ingredientRatings.itertuples(name='rating', index=False):
        recipes_dict[recipe_id].append(ingredient_id) 
        ingredients_dict[user_id][ingredient_id].append(rating)
 
    for key in ingredients_dict.copy().keys():
        for value in ingredients_dict[key].copy().keys():
            ingredients_dict[key][value]  = sum(ingredients_dict[key][value])/len(ingredients_dict[key][value])
    

    
    test_ratings = ratings.rename(columns={'item_id': 'recipe_id'}).iloc[test]
    
    content_based_predictions = defaultdict(lambda: defaultdict())                        
    for user_id, recipe_id, _ in test_ratings.itertuples(name='predictions', index=False):
        rating_est = 0.0
        counter = 0
        for ingredient_id in recipes_dict[recipe_id]:
            if(type(ingredients_dict[user_id][ingredient_id]) == float):
                rating_est += ingredients_dict[user_id][ingredient_id]
                counter += 1
        if(counter != 0):
            content_based_predictions[user_id][recipe_id] = rating_est/counter
        else:
            content_based_predictions[user_id][recipe_id] = np.nan

    test_ratings['rating_est'] = test_ratings.apply(lambda x: content_based_predictions[x.user_id][x.recipe_id], axis=1)
    test_ratings['details'] = "{'is it awesome': Yes}"
    test_ratings.columns = ['uid', 'iid', 'r_ui', 'est', 'details']
    test_ratings = test_ratings.dropna()
    predictions = list(test_ratings.itertuples(name='Prediction', index=False))
    
    return predictions

In [102]:
ingredientRatings

,user_id,recipe_id,ingredient_id,rating_est
0,455,50,20312,4.374446
1,455,50,4397,4.374446
2,455,50,4147,4.374446
3,455,50,5803,4.374446
4,455,50,16241,4.374446
...,...,...,...,...
10759395,78982,3287,7428,4.115247
10759396,78982,3287,7842,4.115247
10759397,78982,3287,1525,4.115247
10759398,78982,3287,18805,4.115247


In [12]:
import random
def get_baseline(predictions):
    predDF = pd.DataFrame.from_records(predictions, columns=['uid', 'iid', 'r_ui', 'est', 'details'])
    predDF['est'] = np.random.randint(1, 6, predDF.shape[0])
    randomized_predictions = list(predDF.itertuples(index=False))
    return randomized_predictions

In [13]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

reader = Reader()
data = Dataset.load_from_df(ratings, reader)
# there is a trade off between NDCG score and recall. Tuning the parameters may increase one and lower the other, or lower both..
param_grid = {'n_epochs': [5,5], 'lr_all': [0.01, 0.1], 'reg_all':[0.01,0.1]}
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=2)
grid_search.fit(data)
print(grid_search.best_params['rmse'])
svd = grid_search.best_estimator['rmse']

{'n_epochs': 5, 'lr_all': 0.01, 'reg_all': 0.1}


In [16]:



kf = KFold(n_splits=5)

precisionAvg = 0
recallAvg = 0
ndcgAvg = 0
healthinessAvg = 0
for trainset, testset in kf.split(data):
    svd.fit(trainset)
    # The second parameter is the post fileter healthiness factor.
    predictions2 = postfilter(svd.test(testset), 0.0)
    baseline
    precision, recall, ndcg, healthinessAverage, top_n2 = evaluations_at_k(predictions2)
    precisionAvg = precisionAvg + precision
    recallAvg = recallAvg + recall
    ndcgAvg = ndcgAvg + ndcg
    healthinessAvg = healthinessAvg + healthinessAverage
    
print("Precision:", precisionAvg/5, "Recall:", recallAvg/5, "NDCG_score:", ndcgAvg/5, "Healthiness:", healthinessAvg/5)

C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=5 as keyword args. From version 0.25 passing these as positional arguments will r

Precision: 0.8800106924773139 Recall: 0.8889711781166509 NDCG_score: 0.9843743315508021 Healthiness: -1.0254081768914558


C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [46]:
predictions2

[Prediction(uid=37936, iid=470, r_ui=4.0, est=4.736381712382758, details={'was_impossible': False}),
 Prediction(uid=68051, iid=470, r_ui=5.0, est=4.705210450081074, details={'was_impossible': False}),
 Prediction(uid=74731, iid=470, r_ui=4.0, est=4.716317517622305, details={'was_impossible': False}),
 Prediction(uid=37267, iid=470, r_ui=5.0, est=4.8002375101281425, details={'was_impossible': False}),
 Prediction(uid=37007, iid=901, r_ui=5.0, est=4.280244833424879, details={'was_impossible': False}),
 Prediction(uid=11567, iid=901, r_ui=5.0, est=4.5516802589052645, details={'was_impossible': False}),
 Prediction(uid=44912, iid=901, r_ui=5.0, est=3.8348399172413306, details={'was_impossible': False}),
 Prediction(uid=51556, iid=901, r_ui=5.0, est=4.459416133380489, details={'was_impossible': False}),
 Prediction(uid=83930, iid=901, r_ui=5.0, est=4.113802452489845, details={'was_impossible': False}),
 Prediction(uid=42539, iid=901, r_ui=5.0, est=4.642779880892305, details={'was_impossibl

In [17]:
#Precision: 0.873187744664541 Recall: 0.9376725207622437 NDCG_score: 0.9683188782350868 Healthiness: 0.9837449819605013

In [115]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
# We use 5 fold cross evaluation.
kf = KFold(n_splits=5)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1) 
        
# Run the algorithm 5 times and evaluate.
for train, test in skf.split(ratings, ratings['rating']):
    predictions1 = content_based_predict(train, test)
    predictions1 = postfilter(predictions1, 0.0)
    precision, recall, ndcg, healthinessAverage, top_n = evaluations_at_k(predictions1)
   
    print("Precision:", precision, "Recall:", recall, "NDCG_score:", ndcg, "Healthiness:", healthinessAverage)

C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Precision: 0.861848852901483 Recall: 0.8558005644639676 NDCG_score: 0.9418803418803419 Healthiness: -0.9021459227467816


C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Precision: 0.8686122448979586 Recall: 0.8700904778030555 NDCG_score: 0.9502183406113537 Healthiness: -0.8680459770114938


C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Precision: 0.8756368563685624 Recall: 0.8591290291345333 NDCG_score: 0.9499999999999998 Healthiness: -0.9423076923076925


C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Precision: 0.8649726775956278 Recall: 0.8619804843816093 NDCG_score: 0.9533039647577092 Healthiness: -0.8919037199124723
Precision: 0.8753430185633573 Recall: 0.862141699012782 NDCG_score: 0.9663461538461537 Healthiness: -0.6853333333333328


C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [98]:
Precision: 0.8754854368932024 Recall: 0.8784345372900033 NDCG_score: 0.9999999999999999 Healthiness: -0.9433962264150949

[Prediction(uid=455, iid=457, r_ui=4.0, est=4.244725024635784, details="{'is it awesome': Yes}"),
 Prediction(uid=14588, iid=457, r_ui=5.0, est=4.675437750803308, details="{'is it awesome': Yes}"),
 Prediction(uid=20131, iid=457, r_ui=4.0, est=4.493841103165523, details="{'is it awesome': Yes}"),
 Prediction(uid=31136, iid=457, r_ui=3.0, est=4.214786260967958, details="{'is it awesome': Yes}"),
 Prediction(uid=35985, iid=457, r_ui=5.0, est=4.519495296399854, details="{'is it awesome': Yes}"),
 Prediction(uid=46657, iid=457, r_ui=4.0, est=4.047904628445858, details="{'is it awesome': Yes}"),
 Prediction(uid=48103, iid=457, r_ui=5.0, est=4.705834467363385, details="{'is it awesome': Yes}"),
 Prediction(uid=54305, iid=457, r_ui=4.0, est=4.356524812051509, details="{'is it awesome': Yes}"),
 Prediction(uid=58505, iid=457, r_ui=4.0, est=3.990394458623508, details="{'is it awesome': Yes}"),
 Prediction(uid=61083, iid=457, r_ui=4.0, est=4.120246793865743, details="{'is it awesome': Yes}"),
 P

In [105]:
predictions1

[Prediction(uid=455, iid=160, r_ui=5.0, est=4.24603174603173, details="{'is it awesome': Yes}"),
 Prediction(uid=5940, iid=160, r_ui=5.0, est=4.19629629629628, details="{'is it awesome': Yes}"),
 Prediction(uid=8034, iid=160, r_ui=5.0, est=4.5549382716049465, details="{'is it awesome': Yes}"),
 Prediction(uid=11559, iid=160, r_ui=5.0, est=4.5, details="{'is it awesome': Yes}"),
 Prediction(uid=14588, iid=160, r_ui=5.0, est=4.715526695526709, details="{'is it awesome': Yes}"),
 Prediction(uid=15264, iid=160, r_ui=5.0, est=4.166666666666672, details="{'is it awesome': Yes}"),
 Prediction(uid=16054, iid=160, r_ui=5.0, est=4.552777777777776, details="{'is it awesome': Yes}"),
 Prediction(uid=17078, iid=160, r_ui=4.0, est=4.768140589569162, details="{'is it awesome': Yes}"),
 Prediction(uid=20340, iid=160, r_ui=5.0, est=3.909644481574268, details="{'is it awesome': Yes}"),
 Prediction(uid=20362, iid=160, r_ui=5.0, est=4.516729797979793, details="{'is it awesome': Yes}"),
 Prediction(uid=220

In [83]:
group = ratings.groupby(by='user_id')

In [ ]:
grps = ratings.groupby('user_id')

train_dict = dict()
test_dict = dict()

for group in grps:
    print(group)
    counter = 0
    for train, test in kf.split(group):
        counter =+ 1
        if counter in train_dict:
            train_dict[counter] = np.concatenate((train_dict[counter], train), axis=1)
            test_dict[counter] = np.concatenate((test_dict[counter], test), axis=1)

In [88]:
group.get_group(455)

,user_id,item_id,rating
0,455,50,3.0
1,455,457,4.0
2,455,28,5.0
3,455,458,3.0
4,455,459,5.0
5,455,153,5.0
6,455,160,5.0
7,455,279,5.0
8,455,461,4.0
9,455,103,5.0


In [86]:
ratings

,user_id,item_id,rating
0,455,50,3.0
1,455,457,4.0
2,455,28,5.0
3,455,458,3.0
4,455,459,5.0
...,...,...,...
50676,84839,131,3.0
50677,84839,109,5.0
50678,84839,145,5.0
50679,84839,133,5.0


In [27]:
    train_ratings = ratings.iloc[train]
    
    users = train_ratings['user_id'].drop_duplicates().to_frame()
    recipes = train_ratings['item_id'].drop_duplicates().to_frame().rename(columns={'item_id': 'recipe_id'})
    users['key'] = 1
    recipes['key'] = 1
    users_recipes = users.merge(recipes, how='outer').drop("key", 1)
    users_recipes['rating_est'] = np.nan
    
    reader = Reader()
    data = Dataset.load_from_df(train_ratings, reader)
    trainset = data.build_full_trainset()
    svd.fit(trainset)
    
    
    users_recipes['rating_est'] = users_recipes.apply(lambda x: svd.predict(x.user_id, x.recipe_id, r_ui=None, clip=True, verbose=False)[3], axis=1)
    
    


    

    


In [49]:
ratings

,user_id,item_id,rating
0,455,50,3.0
1,455,457,4.0
2,455,28,5.0
3,455,458,3.0
4,455,459,5.0
...,...,...,...
50676,84839,131,3.0
50677,84839,109,5.0
50678,84839,145,5.0
50679,84839,133,5.0


In [68]:
foo = ratings[ratings['rating'] > 3].groupby('user_id')['rating'].count()
foo = 10/foo
foo.mean()

0.3804773464169082

In [39]:
   # for user_id, item_id, rating in train_ratings.itertuples(name='ratings', index=False):
    #    users_recipes.loc[('user_id' == user_id) & ('recipe_id' == item_id)].at['rating_est'] = rating
    
        
train_ratings = users_recipes
    
recipe_id_ingredient_id = item_profiles3[['Recipe ID', 'Ingredient ID']]
ingredientRatings = train_ratings.merge(recipe_id_ingredient_id, left_on='recipe_id', right_on='Recipe ID').drop('Recipe ID',1)
ingredientRatings = ingredientRatings[['user_id', 'recipe_id', 'Ingredient ID', 'rating_est']]
ingredientRatings = ingredientRatings.rename(columns={'Ingredient ID': 'ingredient_id', 'item_id': 'recipe_id'})

In [37]:
(users_recipes.user_id == 455).sum()

1031

In [40]:
    recipes_dict = defaultdict(list) 
    ingredients_dict = defaultdict(lambda: defaultdict(list))  
    for user_id, recipe_id, ingredient_id, rating in ingredientRatings.itertuples(name='rating', index=False):
        recipes_dict[recipe_id].append(ingredient_id) 
        ingredients_dict[user_id][ingredient_id].append(rating)
 
    for key in ingredients_dict.copy().keys():
        for value in ingredients_dict[key].copy().keys():
            ingredients_dict[key][value]  = sum(ingredients_dict[key][value])/len(ingredients_dict[key][value])
    


In [41]:
    test_ratings = ratings.rename(columns={'item_id': 'recipe_id'}).iloc[test]
    
    content_based_predictions = defaultdict(lambda: defaultdict())                        
    for user_id, recipe_id, _ in test_ratings.itertuples(name='predictions', index=False):
        rating_est = 0.0
        counter = 0
        for ingredient_id in recipes_dict[recipe_id]:
            if(type(ingredients_dict[user_id][ingredient_id]) == float):
                rating_est += ingredients_dict[user_id][ingredient_id]
                counter += 1
        if(counter != 0):
            content_based_predictions[user_id][recipe_id] = rating_est/counter
        else:
            content_based_predictions[user_id][recipe_id] = np.nan

    test_ratings['rating_est'] = test_ratings.apply(lambda x: content_based_predictions[x.user_id][x.recipe_id], axis=1)
    test_ratings['details'] = "{'is it awesome': Yes}"
    test_ratings.columns = ['uid', 'iid', 'r_ui', 'est', 'details']
    predictions = list(test_ratings.itertuples(name='Prediction', index=False))

In [45]:
test_ratings = ratings.rename(columns={'item_id': 'recipe_id'}).iloc[test]
(test_ratings.user_id == 455).sum()

0

In [ ]:
users_recipes

In [ ]:
ingredientRatings